In [2]:
import pandas as pd   
import numpy as np    
import matplotlib.pyplot as plt 
%matplotlib inline
%config InlineBackend.figure_formats = ['retina']
import seaborn as sns
import time
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, log_loss
from sklearn.metrics import auc, roc_curve, roc_auc_score, precision_recall_curve
from sklearn.metrics import fbeta_score, cohen_kappa_score
SEED = 42

In [5]:
df = pd.read_csv(r"..\Dataset\diabetes.csv")
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '..\\Dataset\\diabetes.csv'

In [12]:
X = df.drop("Outcome", axis=1)  # axis=0 for row, axis=1 for column
y = df["Outcome"]

# split data to 80:20 ratio for train/test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=SEED, stratify=y
)
print("X_train", X_train.shape)
print("y_train", y_train.shape)
print("X_test", X_test.shape)
print("y_test", y_test.shape)

X_train (614, 8)
y_train (614,)
X_test (154, 8)
y_test (154,)


In [13]:
! pip install sklearn_nature_inspired_algorithms==0.4.3
! pip install NiaPy
! pip install algorithms

     ---------------------------------------- 0.0/6.9 MB ? eta -:--:--
     - -------------------------------------- 0.3/6.9 MB 4.5 MB/s eta 0:00:02
     --- ------------------------------------ 0.6/6.9 MB 5.5 MB/s eta 0:00:02
     ----- ---------------------------------- 1.0/6.9 MB 6.3 MB/s eta 0:00:01
     ------ --------------------------------- 1.1/6.9 MB 7.0 MB/s eta 0:00:01
     ------ --------------------------------- 1.1/6.9 MB 4.5 MB/s eta 0:00:02
     ------------ --------------------------- 2.1/6.9 MB 7.0 MB/s eta 0:00:01
     -------------- ------------------------- 2.5/6.9 MB 7.1 MB/s eta 0:00:01
     --------------- ------------------------ 2.7/6.9 MB 7.1 MB/s eta 0:00:01
     --------------- ------------------------ 2.7/6.9 MB 7.1 MB/s eta 0:00:01
     ------------------ --------------------- 3.2/6.9 MB 6.9 MB/s eta 0:00:01
     --------------------- ------------------ 3.7/6.9 MB 7.2 MB/s eta 0:00:01
     ----------------------- ---------------- 4.1/6.9 MB 7.3 MB/s eta 0

  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [1477 lines of output]
      C:\Users\nikhi\AppData\Local\Temp\pip-install-ie5qbfo4\scikit-learn_49c288a90eac47ea90663f7174f94cc3\setup.py:12: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
        from pkg_resources import parse_version
      Partial import of sklearn during the build process.
      C:\Users\nikhi\AppData\Local\Temp\pip-install-ie5qbfo4\scikit-learn_49c288a90eac47ea90663f7174f94cc3\setup.py:123: DeprecationWarning:
      
        `numpy.distutils` is deprecated since NumPy 1.23.0, as a result
        of the deprecation of `distutils` itself. It will be removed for
        Python >= 3.12. For older Python versions it will remain present.
        It is recommended to use `setuptools < 60.0` for those Python versions.
        For more details, see:
          https://numpy.org/devd

  You can safely remove it manually.



   ---------------------------------------- 0.0/183.9 kB ? eta -:--:--
   -------------------- ------------------- 92.2/183.9 kB 5.1 MB/s eta 0:00:01
   ------------------------------------- -- 174.1/183.9 kB 3.5 MB/s eta 0:00:01
   ------------------------------------- -- 174.1/183.9 kB 3.5 MB/s eta 0:00:01
   ------------------------------------- -- 174.1/183.9 kB 3.5 MB/s eta 0:00:01
   ------------------------------------- -- 174.1/183.9 kB 3.5 MB/s eta 0:00:01
   -------------------------------------- 183.9/183.9 kB 793.0 kB/s eta 0:00:00
   ---------------------------------------- 0.0/7.7 MB ? eta -:--:--
    --------------------------------------- 0.2/7.7 MB 3.1 MB/s eta 0:00:03
   - -------------------------------------- 0.3/7.7 MB 3.9 MB/s eta 0:00:02
   -- ------------------------------------- 0.5/7.7 MB 3.9 MB/s eta 0:00:02
   ---- ----------------------------------- 0.8/7.7 MB 4.4 MB/s eta 0:00:02
   ----- ---------------------------------- 1.1/7.7 MB 5.0 MB/s eta 0:00:02


In [14]:
def evaluate(model, X_test, y_test, modelname):
    y_pred1 = model.predict(X_test)
    precision = precision_score(y_test, y_pred1)
    recall = recall_score(y_test, y_pred1)
    f1score = f1_score(y_test, y_pred1)
    rocauc = roc_auc_score(y_test, y_pred1)
    logloss = log_loss(y_test, y_pred1)
    accuracy = accuracy_score(y_test, y_pred1)

    df_model = pd.DataFrame(
        {
            "model": [modelname],
            "accuracy": [accuracy],
            "precision": [precision],
            "recall": [recall],
            "f1score": [f1score],
            "rocauc": [rocauc],
            "logloss": [logloss],
            "timetaken": [time1],
        }
    )
    return df_model

In [15]:
n_estimators = [int(x) for x in np.linspace(200, 2000, num=10)]
# Number of features to consider at every split
max_features = ["auto", "sqrt"]
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num=11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
param_grid_rt = {
    "n_estimators": n_estimators,
    "max_features": max_features,
    "max_depth": max_depth,
    "min_samples_split": min_samples_split,
    "min_samples_leaf": min_samples_leaf,
    "bootstrap": bootstrap,
}
print(param_grid_rt)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


NATURE INSPIRIED SEARCH


In [21]:
from sklearn_nature_inspired_algorithms.model_selection import (
    NatureInspiredSearchCV,
)
from sklearn.ensemble import RandomForestClassifier
import time

# Define the parameter grid for NatureInspiredSearchCV
param_grid_rt = {
    "max_features": ["sqrt", "log2", None]  # Valid options for max_features
}

# Start the timer
time1 = time.time()

# Initialize the RandomForestClassifier
clf_1 = RandomForestClassifier()

# Initialize NatureInspiredSearchCV
nia_search = NatureInspiredSearchCV(
    clf_1,
    param_grid_rt,
    cv=3,
    verbose=2,
    algorithm="ba",  # Bat algorithm
    population_size=50,
    max_n_gen=100,
    max_stagnating_gen=20,
    runs=5,
    n_jobs=-1,
    random_state=42,
    error_score="raise",  # Raise an error if fitting fails
)

# Fit NatureInspiredSearchCV
nia_search.fit(X_train, y_train)

# End the timer and print elapsed time
print("Time taken:", time.time() - time1, "seconds")

# Get the best estimator found during the search
best_estimator = nia_search.best_estimator_

# Evaluate the best estimator
model1 = evaluate(best_estimator, X_test, y_test, "Random Forest")
print(model1)

Fitting 3 folds for some of the 3 candidates, which might total in 9 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Run 1/5, Iteration 1/100 | Stagnation 0/20 | There were 3 candidates trained until now
Run 1/5, Iteration 2/100 | Stagnation 1/20 | There were 3 candidates trained until now
Run 1/5, Iteration 3/100 | Stagnation 2/20 | There were 3 candidates trained until now
Run 1/5, Iteration 4/100 | Stagnation 3/20 | There were 3 candidates trained until now
Run 1/5, Iteration 5/100 | Stagnation 4/20 | There were 3 candidates trained until now
Run 1/5, Iteration 6/100 | Stagnation 5/20 | There were 3 candidates trained until now
Run 1/5, Iteration 7/100 | Stagnation 6/20 | There were 3 candidates trained until now
Run 1/5, Iteration 8/100 | Stagnation 7/20 | There were 3 candidates trained until now
Run 1/5, Iteration 9/100 | Stagnation 8/20 | There we

In [22]:
nia_search.best_params_

{'max_features': None}

In [23]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier()
print(et.get_params())

{'bootstrap': False, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}


In [24]:
n_estimators = [int(x) for x in np.linspace(start=100, stop=2000, num=10)]
# Number of features to consider at every split
max_features = ["auto", "sqrt"]
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num=11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
param_grid_et = {
    "n_estimators": n_estimators,
    "max_features": max_features,
    "max_depth": max_depth,
    "min_samples_split": min_samples_split,
    "min_samples_leaf": min_samples_leaf,
    "bootstrap": bootstrap,
}
print(param_grid_et)

{'n_estimators': [100, 311, 522, 733, 944, 1155, 1366, 1577, 1788, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [26]:
from sklearn_nature_inspired_algorithms.model_selection import (
    NatureInspiredSearchCV,
)
from sklearn.ensemble import ExtraTreesClassifier
import time

# Define the parameter grid for ExtraTreesClassifier
param_grid_et = {
    "max_features": [None, "sqrt", "log2"]  # Valid options for max_features
}

# Start the timer
time1 = time.time()

# Initialize the ExtraTreesClassifier
clf_2 = ExtraTreesClassifier()

# Initialize NatureInspiredSearchCV for ExtraTreesClassifier
nia_search2 = NatureInspiredSearchCV(
    clf_2,
    param_grid_et,
    cv=3,
    verbose=2,
    algorithm="ba",  # Bat algorithm
    population_size=50,
    max_n_gen=100,
    max_stagnating_gen=20,
    runs=5,
    n_jobs=-1,
    random_state=42,
    error_score="raise",  # Raise an error if fitting fails
)

try:
    # Fit NatureInspiredSearchCV for ExtraTreesClassifier
    nia_search2.fit(X_train, y_train)

    # Get the best estimator found during the search for ExtraTreesClassifier
    ba_et = nia_search2.best_estimator_

    # Evaluate the best estimator
    model3 = evaluate(ba_et, X_test, y_test, "Extra Tree")

    # Print the time taken for the fitting process
    print("Time taken:", time.time() - time1, "seconds")

    # Update the time taken in the model evaluation
    model3.timetaken[0] = time.time() - time1

    # Print the evaluation results
    print(model3)

except ValueError as e:
    print("Fitting failed:", e)

Fitting 3 folds for some of the 3 candidates, which might total in 9 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Run 1/5, Iteration 1/100 | Stagnation 0/20 | There were 3 candidates trained until now
Run 1/5, Iteration 2/100 | Stagnation 1/20 | There were 3 candidates trained until now
Run 1/5, Iteration 3/100 | Stagnation 2/20 | There were 3 candidates trained until now
Run 1/5, Iteration 4/100 | Stagnation 3/20 | There were 3 candidates trained until now
Run 1/5, Iteration 5/100 | Stagnation 4/20 | There were 3 candidates trained until now
Run 1/5, Iteration 6/100 | Stagnation 5/20 | There were 3 candidates trained until now
Run 1/5, Iteration 7/100 | Stagnation 6/20 | There were 3 candidates trained until now
Run 1/5, Iteration 8/100 | Stagnation 7/20 | There were 3 candidates trained until now
Run 1/5, Iteration 9/100 | Stagnation 8/20 | There we

In [27]:
nia_search2.best_params_

{'max_features': 'sqrt'}

In [29]:
%%time
from sklearn_nature_inspired_algorithms.model_selection import NatureInspiredSearchCV
import time

# Start the timer
time1 = time.time()

# Define the parameter grid with valid options for 'max_features'
param_grid = {
    'bootstrap': [True],
    'max_depth': [110],
    'max_features': ['sqrt', 'log2'],  # Valid options for 'max_features'
    'min_samples_leaf': [1],
    'min_samples_split': [5],
    'n_estimators': [800]
}

# Initialize the ExtraTreesClassifier
clf_2 = ExtraTreesClassifier()

# Initialize NatureInspiredSearchCV for ExtraTreesClassifier
nia_search2 = NatureInspiredSearchCV(
    clf_2,
    param_grid,
    algorithm='ba',  # Bat algorithm
    population_size=50,
    max_n_gen=100,
    max_stagnating_gen=20,
    runs=3,
    random_state=42
)

try:
    # Fit NatureInspiredSearchCV for ExtraTreesClassifier
    nia_search2.fit(X_train, y_train)
    
    # Get the best estimator found during the search for ExtraTreesClassifier
    ba_et = nia_search2.best_estimator_
    
    # Evaluate the best estimator
    model3 = evaluate(ba_et, X_test, y_test, 'Extra Tree')
    
    # Print the time taken for the fitting process
    print("Time taken:", time.time() - time1, "seconds")
    
    # Update the time taken in the model evaluation
    model3.timetaken[0] = time.time() - time1
    
    # Print the evaluation results
    print(model3)
    
except ValueError as e:
    print("Fitting failed:", e)


Fitting at most 2 candidates
Optimization finished, 2 candidates were fitted
Time taken: 17.54552674293518 seconds
        model  accuracy  precision    recall   f1score    rocauc    logloss  \
0  Extra Tree  0.720779   0.607843  0.574074  0.590476  0.687037  10.064137   

   timetaken  
0  17.545527  
CPU times: total: 13.2 s
Wall time: 17.6 s


In [30]:
nia_search2.best_params_

{'bootstrap': True,
 'max_depth': 110,
 'max_features': 'log2',
 'min_samples_leaf': 1,
 'min_samples_split': 5,
 'n_estimators': 800}

In [31]:
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier()
print(dtc.get_params())

{'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'random_state': None, 'splitter': 'best'}


In [32]:
class_weight = [int(x) for x in np.linspace(start=10, stop=100, num=10)]
class_weight.append(None)
# Number of features to consider at every split
max_features = ["auto", "sqrt"]
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 210, num=11)]
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15]
# Minimum number of samples required at each leaf node
min_samples_leaf = [2, 4, 10]
# Method of selecting samples for training each tree
# Create the random grid
param_grid_dt = {  #'class_weight': class_weight,
    "max_features": max_features,
    "max_depth": max_depth,
    "min_samples_split": min_samples_split,
    "min_samples_leaf": min_samples_leaf,
}
print(param_grid_dt)

{'max_features': ['auto', 'sqrt'], 'max_depth': [10, 30, 50, 70, 90, 110, 130, 150, 170, 190, 210], 'min_samples_split': [2, 5, 10, 15], 'min_samples_leaf': [2, 4, 10]}


In [34]:
%%time
from sklearn_nature_inspired_algorithms.model_selection import NatureInspiredSearchCV
import time

# Start the timer
time1 = time.time()

# Define the parameter grid with valid options for 'max_features'
param_grid_dt = {
    'max_features': ['sqrt', 'log2', None]  # Valid options for 'max_features'
}

# Initialize the DecisionTreeClassifier
clf_3 = DecisionTreeClassifier()

# Initialize NatureInspiredSearchCV for DecisionTreeClassifier
nia_search3 = NatureInspiredSearchCV(
    clf_3,
    param_grid_dt,
    cv=3,
    verbose=2,
    algorithm='ba',  # Bat algorithm
    population_size=50,
    max_n_gen=100,
    max_stagnating_gen=20,
    runs=5,
    n_jobs=-1,
    random_state=42
)

try:
    # Fit NatureInspiredSearchCV for DecisionTreeClassifier
    nia_search3.fit(X_train, y_train)
    
    # Get the best estimator found during the search for DecisionTreeClassifier
    ba_dt = nia_search3.best_estimator_
    
    # Evaluate the best estimator
    model4 = evaluate(ba_dt, X_test, y_test, 'Decision Tree')
    
    # Print the time taken for the fitting process
    print("Time taken:", time.time() - time1, "seconds")
    
    # Update the time taken in the model evaluation
    model4.timetaken[0] = time.time() - time1
    
    # Print the evaluation results
    print(model4)
    
except ValueError as e:
    print("Fitting failed:", e)


Fitting 3 folds for some of the 3 candidates, which might total in 9 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Run 1/5, Iteration 1/100 | Stagnation 0/20 | There were 3 candidates trained until now
Run 1/5, Iteration 2/100 | Stagnation 1/20 | There were 3 candidates trained until now
Run 1/5, Iteration 3/100 | Stagnation 2/20 | There were 3 candidates trained until now
Run 1/5, Iteration 4/100 | Stagnation 3/20 | There were 3 candidates trained until now
Run 1/5, Iteration 5/100 | Stagnation 4/20 | There were 3 candidates trained until now
Run 1/5, Iteration 6/100 | Stagnation 5/20 | There were 3 candidates trained until now
Run 1/5, Iteration 7/100 | Stagnation 6/20 | There were 3 candidates trained until now
Run 1/5, Iteration 8/100 | Stagnation 7/20 | There were 3 candidates trained until now
Run 1/5, Iteration 9/100 | Stagnation 8/20 | There we

In [35]:
nia_search3.best_params_

{'max_features': 'sqrt'}

In [37]:
%%time
from sklearn_nature_inspired_algorithms.model_selection import NatureInspiredSearchCV
import time

# Start the timer
time1 = time.time()

# Define the parameter grid with valid options for 'max_features'
param_grid = {
    'max_depth': [115],
    'max_features': [None],  # Use None instead of 'auto'
    'min_samples_leaf': [4],
    'min_samples_split': [10],
}

# Initialize the DecisionTreeClassifier
clf_3 = DecisionTreeClassifier()

# Initialize NatureInspiredSearchCV for DecisionTreeClassifier
nia_search3 = NatureInspiredSearchCV(
    clf_3,
    param_grid,
    cv=3,
    verbose=2,
    algorithm='ba',  # Bat algorithm
    population_size=150,
    max_n_gen=200,
    max_stagnating_gen=60,
    runs=5,
    n_jobs=-1,
    random_state=42
)

try:
    # Fit NatureInspiredSearchCV for DecisionTreeClassifier
    nia_search3.fit(X_train, y_train)
    
    # Get the best estimator found during the search for DecisionTreeClassifier
    ba_dt = nia_search3.best_estimator_
    
    # Evaluate the best estimator
    model3 = evaluate(ba_dt, X_test, y_test, 'Decision Tree')
    
    # Print the time taken for the fitting process
    print("Time taken:", time.time() - time1, "seconds")
    
    # Update the time taken in the model evaluation
    model3.timetaken[0] = time.time() - time1
    
    # Print the evaluation results
    print(model3)
    
except ValueError as e:
    print("Fitting failed:", e)


Fitting 3 folds for some of the 1 candidates, which might total in 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Run 1/5, Iteration 1/200 | Stagnation 0/60 | There were 1 candidates trained until now
Run 1/5, Iteration 2/200 | Stagnation 1/60 | There were 1 candidates trained until now
Run 1/5, Iteration 3/200 | Stagnation 2/60 | There were 1 candidates trained until now
Run 1/5, Iteration 4/200 | Stagnation 3/60 | There were 1 candidates trained until now
Run 1/5, Iteration 5/200 | Stagnation 4/60 | There were 1 candidates trained until now
Run 1/5, Iteration 6/200 | Stagnation 5/60 | There were 1 candidates trained until now
Run 1/5, Iteration 7/200 | Stagnation 6/60 | There were 1 candidates trained until now
Run 1/5, Iteration 8/200 | Stagnation 7/60 | There were 1 candidates trained until now
Run 1/5, Iteration 9/200 | Stagnation 8/60 | There were 1 candidates trained until now
Run 1/5, Iteration 10/200 | Stagnation 9/60 | There were 1 candidates trained until 

In [38]:
nia_search3.best_params_

{'max_depth': 115,
 'max_features': None,
 'min_samples_leaf': 4,
 'min_samples_split': 10}

In [39]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
print(lr.get_params())

{'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': None, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}


In [40]:
max_iter = [int(x) for x in np.linspace(start=200, stop=2000, num=10)]
C = [float(x) for x in np.linspace(start=1, stop=20, num=20)]
solver = ["newton-cg", "lbfgs", "liblinear", "sag", "saga"]
class_weight = [int(x) for x in np.linspace(1, 10, num=10)]
class_weight.append(None)
n_jobs = [int(x) for x in np.linspace(1, 10, num=10)]
# n_jobs.append(None)
intercept_scaling = [1, 2, 4]
verbose = [0, 1, 2, 3]
dual = [True, False]
multi_class = ["auto", "ovr", "multinomial"]
tol = [0.00001, 0.0001, 0.001, 0.01, 0.1, 0, 1]
# Create the random grid
param_grid_lr = {
    "max_iter": max_iter,
    #'class_weight': class_weight,
    "solver": solver,
    #'intercept_scaling': intercept_scaling,
    #'dual': dual,
    #'n_jobs' : n_jobs,
    #'verbose' : verbose,
    "C": C,
    #'tol' : tol
    #'multi_class' : multi_class
}
print(param_grid_lr)

{'max_iter': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'], 'C': [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0]}


In [41]:
%%time
from sklearn_nature_inspired_algorithms.model_selection import NatureInspiredSearchCV
time1 = time.time()
clf_1 = LogisticRegression()

nia_search1 = NatureInspiredSearchCV(
    clf_1,
    param_grid_lr,
    cv=3,
    verbose=2,
    algorithm='ba', #  bat algorithm
    population_size=50,
    max_n_gen=100,
    max_stagnating_gen=20,
    runs=3,
    n_jobs=-1,
    random_state=42,
    
    # or any number if you want same results on each run
)


nia_search1.fit(X_train, y_train)
print("Bat model")

print("Logistic Regression")

ba_lr= nia_search1.best_estimator_
model2=evaluate(ba_lr,  X_test,  y_test, 'Logistic Regression')
model2.timetaken[0] = time.time() - time1
print(model2)

Fitting 3 folds for some of the 1000 candidates, which might total in 3000 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candi

In [42]:
nia_search1.best_params_

{'max_iter': 1400, 'solver': 'newton-cg', 'C': 20.0}

In [43]:
from sklearn.neighbors import KNeighborsClassifier

kn = KNeighborsClassifier()
print(kn.get_params())

{'algorithm': 'auto', 'leaf_size': 30, 'metric': 'minkowski', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 5, 'p': 2, 'weights': 'uniform'}


In [44]:
leaf_size = [int(x) for x in np.linspace(start=100, stop=200, num=10)]
# n_neighbors=[int(x) for x in np.linspace(start = 10, stop = 100, num = 10)]
# n_neighbors.append(None)
# Number of features to consider at every split
algorithm = ["auto", "ball_tree", "kd_tree", "brute"]
# Maximum number of levels in tree
p = [int(x) for x in np.linspace(1, 20, num=20)]
param_grid_kn = {
    "leaf_size": leaf_size,
    "p": p,
    "algorithm": algorithm,
    #'n_neighbors' : n_neighbors,
}
print(param_grid_kn)

{'leaf_size': [100, 111, 122, 133, 144, 155, 166, 177, 188, 200], 'p': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']}


In [46]:
%%time
from sklearn_nature_inspired_algorithms.model_selection import NatureInspiredSearchCV
import time

time1 = time.time()

clf_4 = KNeighborsClassifier()

nia_search4 = NatureInspiredSearchCV(
    clf_4,
    param_grid_kn,
    cv=3,
    verbose=2,
    algorithm='ba',
    population_size=50,
    max_n_gen=100,
    max_stagnating_gen=20,
    runs=3,
    n_jobs=-1,
    random_state=42
)

try:
    nia_search4.fit(X_train, y_train)
    ba_kn = nia_search4.best_estimator_
    model4 = evaluate(ba_kn, X_test, y_test, 'KNN')
    model4.timetaken[0] = time.time() - time1
    print(model4)
except AttributeError as e:
    print("An error occurred during fitting:", e)


Fitting 3 folds for some of the 800 candidates, which might total in 2400 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candid

In [52]:
from sklearn_nature_inspired_algorithms.model_selection import (
    NatureInspiredSearchCV,
)
import time
from sklearn.neighbors import KNeighborsClassifier

# Define the reduced parameter grid
param_grid = {"leaf_size": [30, 60, 90], "p": [1, 2]}

# Start timing
time1 = time.time()

# Create the KNeighborsClassifier instance
clf_4 = KNeighborsClassifier()

# Initialize NatureInspiredSearchCV
nia_search4 = NatureInspiredSearchCV(
    clf_4,
    param_grid,
    cv=3,
    verbose=2,
    algorithm="ba",  # Bat algorithm
    population_size=50,
    max_n_gen=100,
    max_stagnating_gen=20,
    runs=3,
    n_jobs=-1,
    random_state=42,
)

# Fit the search
nia_search4.fit(X_train, y_train)

# Print the results
print("Bat model")
print("KNN")

# Retrieve the best estimator
ba_kn = nia_search4.best_estimator_

# Evaluate the model
model4 = evaluate(ba_kn, X_test, y_test, "KNN")

# Measure the time taken
model4.timetaken[0] = time.time() - time1

# Print the evaluation results
print(model4)

Fitting 3 folds for some of the 6 candidates, which might total in 18 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Run 1/3, Iteration 1/100 | Stagnation 0/20 | There were 6 candidates trained until now
Run 1/3, Iteration 2/100 | Stagnation 1/20 | There were 6 candidates trained until now
Run 1/3, Iteration 3/100 | Stagnation 2/20 | There were 6 candidates trained until now
Run 1/3, Iteration 4/100 | Stagnation 3/20 | There were 6 candidates trained until now
Run 1/3, Iteration 5/100 | Stagnation 4/20 | There were 6 candidates trained until now
Run 1/3, Iteration 6/100 | Stagnation 5/20 | There were 6 candidates trained until now
Run 1/3, Iteration 7/100 | Stagnation 6/20 | Ther

In [53]:
from sklearn.svm import LinearSVC

lsvc = LinearSVC()
print(lsvc.get_params())

{'C': 1.0, 'class_weight': None, 'dual': 'warn', 'fit_intercept': True, 'intercept_scaling': 1, 'loss': 'squared_hinge', 'max_iter': 1000, 'multi_class': 'ovr', 'penalty': 'l2', 'random_state': None, 'tol': 0.0001, 'verbose': 0}


In [54]:
class_weight = [int(x) for x in np.linspace(start=10, stop=110, num=11)]
class_weight.append(None)
intercept_scaling = [int(x) for x in np.linspace(1, 50, num=10)]
# Minimum number of samples required to split a node
tol = [0.00001, 0.001, 0.0001, 0.01, 0.1, 1]
dual = [True, False]
itr = [int(x) for x in np.linspace(100, 2000, num=20)]
# Create the random grid
param_grid_lsvc = {  #'class_weight':class_weight,
    "tol": tol,
    "intercept_scaling": intercept_scaling,
    "dual": dual,
    "max_iter": itr,
}
print(param_grid_lsvc)

{'tol': [1e-05, 0.001, 0.0001, 0.01, 0.1, 1], 'intercept_scaling': [1, 6, 11, 17, 22, 28, 33, 39, 44, 50], 'dual': [True, False], 'max_iter': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000]}


In [55]:
%%time
from sklearn_nature_inspired_algorithms.model_selection import NatureInspiredSearchCV
time1 = time.time()
clf_5 = LinearSVC()

nia_search5 = NatureInspiredSearchCV(
    clf_5,
    param_grid_lsvc,
    cv=3,
    verbose=2,
    algorithm='ba', #  bat algorithm
    population_size=50,
    max_n_gen=100,
    max_stagnating_gen=20,
    runs=3,
    n_jobs=-1,
    random_state=42,
    
    # or any number if you want same results on each run
)


nia_search5.fit(X_train, y_train)
print("Bat model")

print("LinearSVC")

ba_lsvc= nia_search5.best_estimator_
model5=evaluate(ba_lsvc , X_test,  y_test, 'LinearSVC')
model5.timetaken[0] = time.time() - time1
print(model5)

Fitting 3 folds for some of the 2400 candidates, which might total in 7200 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candi

In [56]:
 nia_search5.best_params_

{'tol': 1e-05, 'intercept_scaling': 50, 'dual': False, 'max_iter': 1600}

In [57]:
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB

gnb = GaussianNB()
print(gnb.get_params())

{'priors': None, 'var_smoothing': 1e-09}


In [58]:
priors = [X_train, y_train]
priors.append(None)
var_smoothing = [
    0.0000000001,
    0.000001,
    0.01,
    0.001,
    0.00000000000001,
    1,
    1e-15,
    1e-20,
]
param_grid_gnb = {"var_smoothing": var_smoothing}  #'priors':priors,
print(param_grid_gnb)

{'var_smoothing': [1e-10, 1e-06, 0.01, 0.001, 1e-14, 1, 1e-15, 1e-20]}


In [59]:
%%time
from sklearn_nature_inspired_algorithms.model_selection import NatureInspiredSearchCV
time1 = time.time()
clf_6 = GaussianNB()

nia_search6 = NatureInspiredSearchCV(
    clf_6,
    param_grid_gnb,
    cv=3,
    verbose=2,
    algorithm='ba', #  bat algorithm
    population_size=50,
    max_n_gen=100,
    max_stagnating_gen=20,
    runs=3,
    n_jobs=-1,
    random_state=42,
    
    # or any number if you want same results on each run
)


nia_search6.fit(X_train, y_train)
print("Bat model")

print("GaussianNB")

ba_gnb= nia_search6.best_estimator_
model6=evaluate(ba_gnb , X_test,  y_test, 'GaussianNB')
model6.timetaken[0] = time.time() - time1
print(model6)

Fitting 3 folds for some of the 8 candidates, which might total in 24 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Run 1/3, Iteration 1/100 | Stagnation 0/20 | There were 8 candidates trained until now
Run 1/3, Iteration 2/100 | Stagnation 1/20 | There were 8 candidates trained until now
Run 1/3, Iteration 3/100 | Stagnation 2/20 | There were 8 candidates trained until now
Run 1/3, Iteration 4/100 | Stagnation 3/20 | There were 8 candidates trained until now
Run 1/3, Iteration 5/100 | Stagnation 4/20 | There were 8 candidates trained until now
Run 1/3, Iteration

In [60]:
 nia_search6.best_estimator_

GaussianNB(var_smoothing=1e-15)

In [61]:
from sklearn.svm import SVC

svc = SVC()
print(svc.get_params())

{'C': 1.0, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 'scale', 'kernel': 'rbf', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}


In [62]:
degree = [int(x) for x in np.linspace(start=1, stop=20, num=20)]
# class_weight.append(None)
cache_size = [int(x) for x in np.linspace(200, 2000, num=10)]
# Minimum number of samples required to split a node
tol = [0.00001, 0.001, 0.0001, 0.01, 0.1, 1]
break_ties = [True, False]
# Create the random grid
param_grid_svc = {
    "degree": degree,
    "tol": tol,
    "cache_size": cache_size,
    "break_ties": break_ties,
}
print(param_grid_svc)

{'degree': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 'tol': [1e-05, 0.001, 0.0001, 0.01, 0.1, 1], 'cache_size': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'break_ties': [True, False]}


In [63]:
%%time
from sklearn_nature_inspired_algorithms.model_selection import NatureInspiredSearchCV
time1 = time.time()
clf_8 = SVC()

nia_search8 = NatureInspiredSearchCV(
    clf_8,
    param_grid_svc,
    cv=3,
    verbose=2,
    algorithm='ba', #  bat algorithm
    population_size=50,
    max_n_gen=100,
    max_stagnating_gen=20,
    runs=3,
    n_jobs=-1,
    random_state=42,
    
    # or any number if you want same results on each run
)


nia_search8.fit(X_train, y_train)
print("Bat model")

print("SVC")

ba_svc= nia_search8.best_estimator_
model8=evaluate(ba_svc , X_test,  y_test, 'SVC')
model8.timetaken[0] = time.time() - time1
print(model8)

Fitting 3 folds for some of the 2400 candidates, which might total in 7200 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candi

In [64]:
 nia_search8.best_params_

{'degree': 16, 'tol': 0.0001, 'cache_size': 1800, 'break_ties': False}